In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import iris
plt.rcParams['font.sans-serif']=['Arial']

In [2]:
Y = 5
lon = 720
lat = 360

In [3]:
res_lon = 360/lon
res_lat = 180/lat
sample_points = [('longitude', np.linspace(0+res_lon/2, 360-res_lon/2, lon)),
                 ('latitude',  np.linspace(-90+res_lat/2, 90-res_lat/2, lat))]

In [4]:
t = Y*12 ## Y years correspond to 12Y months 

lon_range = np.linspace(0+res_lon/2, 360-res_lon/2, lon)
lat_range = np.linspace(-90+res_lat/2, 90-res_lat/2, lat) 
t_range   = np.linspace(1, t, t) 
lat_mesh, t_mesh, lon_mesh = np.meshgrid(lat_range, t_range, lon_range) 

t_grid   = t_mesh.reshape(t*lon*lat, 1)
lon_grid = lon_mesh.reshape(t*lon*lat, 1)
lat_grid = lat_mesh.reshape(t*lon*lat, 1)

dataset = pd.DataFrame(lon_grid, columns = {'longitude'})
dataset['latitude'] = lat_grid
dataset['t'] = t_grid
dataset['loc'] = dataset['longitude']*1000+dataset['latitude']
dataset

,longitude,latitude,t,loc
0,0.25,-89.75,1.0,160.25
1,0.75,-89.75,1.0,660.25
2,1.25,-89.75,1.0,1160.25
3,1.75,-89.75,1.0,1660.25
4,2.25,-89.75,1.0,2160.25
...,...,...,...,...
15551995,357.75,89.75,60.0,357839.75
15551996,358.25,89.75,60.0,358339.75
15551997,358.75,89.75,60.0,358839.75
15551998,359.25,89.75,60.0,359339.75


In [5]:
data = iris.load_cube('/Volumes/Database/CMIP6/MOHC/historical/oh_AERmon_UKESM1-0-LL_historical_r1i1p1f2_gn_200001-201412.nc', 'oh')
time_seq, layer, latitude, longitude = data.shape
data = data[time_seq-t:time_seq,0,:,:]
data = data.interpolate(sample_points, iris.analysis.Linear())
values = np.array(data.data)
oh = values.reshape(t*lon*lat,1)
dataset['oh'] = oh

In [6]:
data = iris.load_cube('/Volumes/Database/Research/ozone-budget/bc179_CH3O2_2010-2014.nc', 'mass_fraction_of_methyl_peroxy_radical_in_air')
time_seq, layer, latitude, longitude = data.shape
data = data[time_seq-t:time_seq,0,:,:]
data = data.interpolate(sample_points, iris.analysis.Linear())
values = np.array(data.data)
ch3o2 = values.reshape(t*lon*lat,1)
dataset['ch3o2_mod'] = ch3o2

In [7]:
data = iris.load_cube('/Volumes/Database/Research/ozone-budget/bc179_HO2_2010-2014.nc', 'mass_fraction_of_hydroperoxyl_radical_in_air')
time_seq, layer, latitude, longitude = data.shape
data = data[time_seq-t:time_seq,0,:,:]
data = data.interpolate(sample_points, iris.analysis.Linear())
values = np.array(data.data)
ho2 = values.reshape(t*lon*lat,1)
dataset['ho2_mod'] = ho2

In [8]:
data = iris.load_cube('/Volumes/Database/Research/ozone-budget/bc179_O1D_2010-2014.nc', 'mass_fraction_of_o1d_in_air')
time_seq, layer, latitude, longitude = data.shape
data = data[time_seq-t:time_seq,0,:,:]
data = data.interpolate(sample_points, iris.analysis.Linear())
values = np.array(data.data)
o1d = values.reshape(t*lon*lat,1)
dataset['o1d_mod'] = o1d

In [10]:
data = iris.load_cube('/Volumes/Database/CMIP6/MOHC/historical/tas_Amon_UKESM1-0-LL_historical_r1i1p1f2_gn_195001-201412.nc', 'tas')
time_seq, latitude, longitude = data.shape
data = data[time_seq-t:time_seq,:,:]
data = data.interpolate(sample_points, iris.analysis.Linear())
values = np.array(data.data)
tas = values.reshape(t*lon*lat,1)
dataset['tas'] = tas

In [12]:
dataset['oh_mole']    = dataset['oh']*1.01325*10**5/dataset['tas']*7.243*10**7*10**9
dataset['ho2_mole']   = dataset['ho2_mod']*1.01325*10**5/dataset['tas']*7.243*10**7*10**9
dataset['ch3o2_mole'] = dataset['ch3o2_mod']*1.01325*10**5/dataset['tas']*7.243*10**7*10**9
dataset['o1d_mole']   = dataset['o1d_mod']*1.01325*10**5/dataset['tas']*7.243*10**7*10**9

In [13]:
dataset_pt1 = dataset[dataset['longitude'] < 180]
dataset_pt2 = dataset[dataset['longitude'] >= 180]
dataset_pt2['longitude'] = dataset_pt2['longitude']-360
dataset_adj = dataset_pt1.append(dataset_pt2)
dataset_adj

,longitude,latitude,t,loc,oh,ch3o2_mod,ho2_mod,o1d_mod,tas,oh_mole,ho2_mole,ch3o2_mole,o1d_mole
0,0.25,-89.75,1.0,160.25,2.290545e-14,2.621025e-12,4.208093e-12,3.342570e-22,249.177536,674629.142382,1.239400e+08,7.719646e+07,0.009845
1,0.75,-89.75,1.0,660.25,2.290431e-14,2.621441e-12,4.207913e-12,3.342296e-22,249.185349,674574.363162,1.239308e+08,7.720628e+07,0.009844
2,1.25,-89.75,1.0,1160.25,2.290404e-14,2.621760e-12,4.207997e-12,3.341813e-22,249.191071,674550.940395,1.239304e+08,7.721391e+07,0.009842
3,1.75,-89.75,1.0,1660.25,2.290429e-14,2.622022e-12,4.208239e-12,3.341204e-22,249.195541,674546.272064,1.239354e+08,7.722023e+07,0.009840
4,2.25,-89.75,1.0,2160.25,2.290455e-14,2.622283e-12,4.208481e-12,3.340595e-22,249.200012,674541.603900,1.239403e+08,7.722653e+07,0.009838
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15551995,-2.25,89.75,60.0,357839.75,4.941682e-18,5.593346e-14,5.392361e-15,0.000000e+00,242.582077,149.503440,1.631381e+05,1.692186e+06,0.000000
15551996,-1.75,89.75,60.0,358339.75,4.939460e-18,5.625231e-14,5.410102e-15,0.000000e+00,242.579269,149.437952,1.636767e+05,1.701852e+06,0.000000
15551997,-1.25,89.75,60.0,358839.75,4.937238e-18,5.657117e-14,5.427844e-15,0.000000e+00,242.576462,149.372462,1.642154e+05,1.711518e+06,0.000000
15551998,-0.75,89.75,60.0,359339.75,4.995137e-18,5.640556e-14,5.470570e-15,0.000000e+00,242.573486,151.125992,1.655100e+05,1.706529e+06,0.000000


In [14]:
radicals_map = dataset_adj.groupby(['loc']).mean()
radicals_map

,longitude,latitude,t,oh,ch3o2_mod,ho2_mod,o1d_mod,tas,oh_mole,ho2_mole,ch3o2_mole,o1d_mole
loc,,,,,,,,,,,,
160.25,0.25,-89.75,30.5,6.812455e-15,9.360089e-13,1.476824e-12,1.390203e-22,227.209686,204102.564441,4.465576e+07,2.862646e+07,0.004184
160.75,0.25,-89.25,30.5,6.765478e-15,9.402426e-13,1.468510e-12,1.394895e-22,227.606094,202343.186519,4.433145e+07,2.870103e+07,0.004192
161.25,0.25,-88.75,30.5,6.718502e-15,9.444761e-13,1.460195e-12,1.399586e-22,228.002502,200584.542087,4.400719e+07,2.877441e+07,0.004201
161.75,0.25,-88.25,30.5,6.671525e-15,9.487096e-13,1.451880e-12,1.404277e-22,228.398926,198826.617121,4.368299e+07,2.884660e+07,0.004209
162.25,0.25,-87.75,30.5,6.745070e-15,9.635654e-13,1.461144e-12,1.425203e-22,228.786880,200750.110478,4.390631e+07,2.925412e+07,0.004267
...,...,...,...,...,...,...,...,...,...,...,...,...
359837.75,-0.25,87.75,30.5,8.590424e-15,1.186136e-12,1.632726e-12,5.531466e-23,254.723450,234569.340043,4.462132e+07,3.236220e+07,0.001510
359838.25,-0.25,88.25,30.5,8.800499e-15,1.163882e-12,1.626700e-12,5.494225e-23,254.622620,240358.337713,4.445369e+07,3.175101e+07,0.001499
359838.75,-0.25,88.75,30.5,8.966818e-15,1.154073e-12,1.621642e-12,5.479602e-23,254.533905,244872.433878,4.431039e+07,3.148412e+07,0.001495


In [15]:
radicals_map.to_csv('/Users/csuen/Documents/GitHub/ozone-radical/Fig. 1/Fig. 1 radicals map.csv', index=False)